In [ ]:
import numpy as np
import tensorflow as tf
import sys
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization,Conv2DTranspose,Concatenate,Input,Activation,Flatten,Dense
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from  tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import torch
# from scipy.io import loadmat
# import h5py
import cv2
import os

In [ ]:
!pip install tqdm
!pip install scikit-learn
!pip install "tensorflow==2.10"
!pip install "numpy<2" --force-reinstall
!pip install matplotlib
!pip install torch
!pip install opencv-python

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
tf.debugging.set_log_device_placement(True)

In [ ]:
dataset = "D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1"

In [ ]:
image_files=sorted(os.listdir('D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1\images'))
mask_files=sorted(os.listdir('D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1\masks'))

In [ ]:
print(image_files[:10])
print(mask_files[:10])

In [ ]:
def display_image_mask(image_path,mask_path):
    count=0
    for image_file,mask_file in zip(image_files,mask_files):
        if count>=6:
            break
        image_path_full=os.path.join(image_path,image_file)
        mask_path_full=os.path.join(mask_path,mask_file)

        image=cv2.imread(image_path_full)
        mask=cv2.imread(mask_path_full,cv2.IMREAD_GRAYSCALE)

        if image is None:
                raise ValueError(f"Image at path {imagepath_full} could not be loaded")
        if mask is None:
                raise ValueError(f"Mask at path {maskpath_full} could not be loaded")

        mask_3ch=cv2.merge([mask,mask,mask])
        image_rgb=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10,10))

        plt.subplot(1, 3, 1)
        plt.imshow(image)
        plt.title('Image')

        plt.subplot(1, 3, 2)
        plt.imshow(mask)
        plt.title('Mask')

        plt.subplot(1,3,3)
        overlay = cv2.addWeighted(image_rgb, 0.7, mask_3ch, 0.3, 0)
        plt.imshow(overlay)
        plt.title("Image with overlay")

        plt.show()
        count+=1

In [ ]:
image_path='D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1\images'
mask_path='D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1\masks'

In [ ]:
display_image_mask(image_path,mask_path)


In [ ]:
images_size=[]
masks_size=[]

for image_file, mask_file in tqdm(zip(image_files, mask_files)):


    imagepath_full = os.path.join(image_path, image_file)
    maskpath_full = os.path.join(mask_path, mask_file)

    image = cv2.imread(imagepath_full)
    mask = cv2.imread(maskpath_full)

    images_size.append(image.shape)
    masks_size.append(mask.shape)

In [ ]:
print(list(set(images_size)))
print(list(set(masks_size)))

In [ ]:
SIZE=224
CHANNEL=1

In [ ]:
def preprocessing_images_masks(num_samples=None):
    images=[]
    masks=[]

    if num_samples is not None:
        selected_image_files = image_files[:num_samples]
        selected_mask_files = mask_files[:num_samples]
    else:
        selected_image_files = image_files
        selected_mask_files = mask_files

    for image_file, mask_file in tqdm(zip(selected_image_files, selected_mask_files)):
        imagepathfull=os.path.join(image_path,image_file)
        maskpathfull=os.path.join(mask_path,mask_file)

        image = cv2.imread(imagepathfull,cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(SIZE,SIZE))
        image=cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
        image = image/255.0
        images.append(image)

        mask = cv2.imread(maskpathfull,cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask,(SIZE,SIZE))
        mask = mask/255.0
        masks.append(mask)

    return np.array(images), np.expand_dims(np.array(masks), axis=-1)